In [ ]:
import torch
import torch.nn as nn
from torch.nn.init import normal_

torch_activation_dict={
'identity':lambda x:x,
'sigmoid':torch.sigmoid,
'relu':torch.relu,
'ranh':torch.tanh,
}

class MLP_torch(nn.Module):
    def __init__(
        self,
        layer_sizes,
        use_bias=True,
        activation='relu',
        output_activation='identity',):
        super().__init__()
        self.activation = torch_activation_dict[activation]
        self.output_activation = torch_activation_dict[output_activation]
        self.layers = nn.ModuleList()
        num_in=layer_sizes[0]
        for num_out in layer_sizes[1:]:
            self.layers.append(nn.Linear(num_in,num_out,bias=use_bias))#默认Kaiming初始化
            # 正态分布初始化，采用与前面手动实现时相同的方式
            normal_(self.layers[-1].weight, std=1.0)
            # 偏置项为全0
            self.layers[-1].bias.data.fill_(0.0)
            num_in = num_out
    def forward(self, x):
        for layer in self.layers[:-1]:
            x=layer(x)
            x=self.activation(x)

        x=self.layers[-1](x)
        x=self.output_activation(x)
        return x



In [ ]:
num_epochs = 1000
learning_rate = 0.1
batch_size = 128
eps = 1e-7
torch.manual_seed(0)

mlp=MLP_torch(layer_sizes=[2,4,1],use_bias=True,out_activation='sigmoid')
optimizer = torch.optim.SGD(mlp.parameters(), lr=learning_rate)

losses = []
test_losses= []
test_accs = []

for epoch in range(num_epochs):
    st=0
    loss=[]
    while True:
        ed=min(st+batch_size,len(x_train))
        if st>=ed:
            break
        x=torch.tensor(x_train[st:ed],dtype=torch.float32)
        y=torch.tensor(y_train[st:ed],dtype=torch.float32)
        y_pred = mlp(x)
        train_loss=torch.mean(-y * torch.log(y_pred + eps) \
            - (1 - y) * torch.log(1 - y_pred + eps))
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        loss.append(train_loss.detach().numpy())
        st+= batch_size

    losses.append(sum(loss)/len(loss))
    with torch.inference_mode():
        x = torch.tensor(x_test, dtype=torch.float32)
        y = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)
        y_pred = mlp(x)
        test_loss = torch.sum(-y * torch.log(y_pred + eps) \
            - (1 - y) * torch.log(1 - y_pred + eps)) / len(x_test)
        test_acc = torch.sum(torch.round(y_pred) == y) / len(x_test)
        test_losses.append(test_loss.detach().numpy())
        test_accs.append(test_acc.detach().numpy())

print('测试精度：', test_accs[-1])
# 将损失变化进行可视化
plt.figure(figsize=(16, 6))
plt.subplot(121)
plt.plot(losses, color='blue', label='train loss')
plt.plot(test_losses, color='red', ls='--', label='test loss')
plt.xlabel('Step')
plt.ylabel('Loss')
plt.title('Cross-Entropy Loss')
plt.legend()

plt.subplot(122)
plt.plot(test_accs, color='red')
plt.ylim(top=1.0)
plt.xlabel('Step')
plt.ylabel('Accuracy')
plt.title('Test Accuracy')
plt.show()


